# Lectura 56: Gráficas en Polars con hvPlot

## Gráficas

Los datos de un `DataFrame` de Polars se pueden visualizar utilizando las librerías de visualización comunes como hvPlot, Matplotlib, Seaborn o Plotly por poner algunos ejemplos.

Polars tiene un método `plot` que nos permite crear gráficos interactivos utilizando la librería `hvPlot`. Lo primero que deberemos realizar para poder crear las gráficas que a continuación mostraremos y muchas otras que esta librería nos permite realizar, es instalar esta librería con `pip install hvplot`.

In [ ]:
# Descomentar y ejecutar la siguiente línea si no se tiene instalada la librería hvPlot
# !pip install hvplot 

In [ ]:
import polars as pl
from polars import col

vuelos_lf = pl.scan_parquet('./data/vuelos/vuelos.parquet')

aerolinea_vs_tiempo_aire = vuelos_lf.group_by('AIRLINE').agg(
    AVG_AIR_TIME=col('AIR_TIME').mean()
).collect()

vuelos_AA_DL = vuelos_lf.filter(
    (col('AIRLINE') == 'AA') | (col('AIRLINE') == 'DL')
).collect()

hexagonal_data = vuelos_lf.select(
    col('AIR_TIME'),
    col('ARRIVAL_DELAY')
).collect()

departure_delay_day = vuelos_lf.select(
    col('DAY'),
    col('AIRLINE'),
    col('DEPARTURE_DELAY')
).collect()

## Gráfico de barras

In [ ]:
aerolinea_vs_tiempo_aire.plot.bar(
    x='AIRLINE',
    y='AVG_AIR_TIME',
    xlabel='Aerolínea',
    ylabel='Promedio tiempo en el aire',
    title='Promedio de tiempo en el aire por aerolínea'
)

## Varios gráficos

Con el parámetro `subplots` podemos separar en gráficas diferentes las dos aerolíneas como se muestra a continuación.

In [ ]:
vuelos_AA_DL.plot.hist(
    'DEPARTURE_DELAY',
    by='AIRLINE',
    bins=20,
    bin_range=(-20,100),
    width=300,
    subplots=True
)

## Gráfico de dispersión

Para agrupar los datos por una o más columnas adicionales, podemos especificar una variable adicional a través del parámetro `by`. Como ejemplo, graficaremos el retraso de salida (`DEPARTURE_DELAY`) en función de la distancia(`DISTANCE`), agrupando los datos por la aerolínea(`AIRLINE`).

In [ ]:
vuelos_AA_DL.plot.scatter(
    x='DISTANCE',
    y='DEPARTURE_DELAY',
    by='AIRLINE',
    alpha=0.2,
    persist=True
)

## Gráfico de bins hexagonales

Podemos crear gráficos de bins hexagonales con el método `hexbin`. Las gráficas `hexbin` pueden ser una alternativa útil a los diagramas de dispersión si nuestros datos son demasiado densos para graficarlos cada punto individualmente. Dado que estos datos no se distribuyen regularmente, usaremos la opción `logz=True` para asignar el eje z (color) a una barra de colores de escala logarítmica.

In [ ]:
hexagonal_data.plot.hexbin(
    x='AIR_TIME',
    y='ARRIVAL_DELAY',
    width=600,
    height=500,
    logz=True
)

## Gráfico de mapa de calor

Un gráfico de mapa de calor nos permite ver la relación entre tres variables, por lo que especificamos las variables `x` e `y` y una variable `C` adicional. Además, podemos definir una función de reducción(`reduce_function` que calcula los valores para cada bin a partir de las muestras que se encuentran en él.

En el siguiente ejemplo graficaremos el retraso en la salida(`DEPARTURE_DELAY`) para cada día del mes(`DAY`) y aerolínea(`AIRLINE`).

In [ ]:
import numpy as np

departure_delay_day.plot.heatmap(
    x='DAY',
    y='AIRLINE',
    C='DEPARTURE_DELAY',
    reduce_function=np.mean,
    colorbar=True
)

Hasta aquí la lista de gráficos que veremos. Es importante destacar que esta lista es solo un subconjunto de todos los tipos de gráficos que podemos realizar a partir de un `DataFrame` de Polars. Si desean ver la lista completa de gráficos disponibles pueden visitar la página de hvPlot.

[Página de hvPlot](https://hvplot.holoviz.org/index.html)